The idea here is to use the multiplex sample as replication (or negation) sample for the CNV findings in the simplex cohort. In other words, we want to check that whatever we found in the simplex cohort is not true for the multiplex cohort. So, no samples in the simplex cohort should be used in the multiplex findings. For that, I'll use the samples_multiplex_only.txt, which means that the samples are only in multiplex and not in simplex (151 samples raw).

Let's start the preprocessing then. First, make sure that no weird samples need to be thrown out:

In [1]:
import glob
ped_files = ['/data/sudregp/multiplex_simplex/multiplex.ped']
wes_prefix = ['CLIA', 'CCGO', 'WPS']
fid = open('/data/sudregp/cnv/xhmm_clean2/samples.txt', 'r')
exclude_list = [line.rstrip() for line in fid]
fid.close()

trios = []
affected = []
controls = []
samples = []
famids = []
for ped_file in ped_files:
    fid = open(ped_file, 'r')
    for line in fid:
        famid, sid, fa, mo, sex, aff = line.rstrip().split('\t')
        # if the current ID and its parents have WES data, and the sample is 
        # not in yet
        if (fa.split('_')[0] in wes_prefix and
            mo.split('_')[0] in wes_prefix and
            sid.split('_')[0] in wes_prefix and
            sid not in samples and
            (sid not in exclude_list or fa not in exclude_list or mo not in exclude_list)):
            fam = {}
            fam['child'] = sid
            if aff == '1':
                affected.append(sid)
            else:
                controls.append(sid)
            fam['father'] = fa
            fam['mother'] = mo
            fam['famid'] = famid
            trios.append(fam)
            samples += [sid, fa, mo]
            famids.append(famid)
    fid.close()
samples = set(samples)
famids = set(famids)
kids = set(affected + controls)

print 'Unique samples:', len(samples)
print 'Unique families:', len(famids)
print 'Unique children:', len(kids)

Unique samples: 153
Unique families: 20
Unique children: 80


So, basically we'll looking at everyone in the multiplex pedigree, provided they're not in the simplex analysis and have parents.

In [3]:
fid = open('/data/sudregp/multiplex_simplex/xhmm_replication/samples.txt', 'w')
for s in samples:
    fid.write(s + '\n')
fid.close()

fid = open('/data/sudregp/multiplex_simplex/xhmm_replication/kid_samples.txt', 'w')
for s in kids:
    fid.write(s + '\n')
fid.close()

Starting a very similar analysis we did for xhmm_ximplex_clean of today's date:

In [ ]:
# in terminal
exome_targets='/data/NCR_SBRB/simplex/SeqCapEZ_Exome_v3.0_Design_Annotation_files/SeqCap_EZ_Exome_v3_hg19_capture_targets.bed'
gatk_memory="50g"
ref_fa='/fdb/GATK_resource_bundle/hg19-2.8/ucsc.hg19.fasta'
out_dir='/data/sudregp/multiplex_simplex/xhmm_replication/'

cd $out_dir
module load GATK
module load XHMM

GATK -m ${gatk_memory} GCContentByInterval -L ${exome_targets} -R ${ref_fa} -o ./DATA.locus_GC.txt
cat ./DATA.locus_GC.txt | awk '{if ($2 < 0.1 || $2 > 0.9) print $1}' > ./extreme_gc_targets.txt

# merging all subjects in the directory
while read s; do
    cp ../xhmm/${s}* .
done < samples.txt
ls -1 *.sample_interval_summary > depth_list.txt;
cp ../xhmm/params.txt .

xhmm --mergeGATKdepths --GATKdepthsList=depth_list.txt -o ./DATA.RD.txt;

# this does the same thing as the XHMM script, but it actually works in parsing 
# the base pair start and ends
cat ${exome_targets} | awk 'BEGIN{OFS="\t"; print "#CHR\tBP1\tBP2\tID"}{print $1, $2, $3, NR}' > ./EXOME.targets.reg

module load plinkseq
pseq . loc-load --locdb ./EXOME.targets.LOCDB --file ./EXOME.targets.reg --group targets \
    --out ./EXOME.targets.LOCDB.loc-load --noweb

# this has the same effect as the suggested command, but it actually works
pseq . loc-stats --locdb ./EXOME.targets.LOCDB --group targets --seqdb ./seqdb.hg19 --noweb | \
    awk '{if (NR > 1) { print  $4, $10 }}' | sed 's/\.\./-/' - > ./DATA.locus_complexity.txt

cat ./DATA.locus_complexity.txt | awk '{if ($2 > 0.25) print $1}' > ./low_complexity_targets.txt

xhmm --matrix -r ./DATA.RD.txt --centerData --centerType target \
-o ./DATA.filtered_centered.RD.txt \
--outputExcludedTargets ./DATA.filtered_centered.RD.txt.filtered_targets.txt \
--outputExcludedSamples ./DATA.filtered_centered.RD.txt.filtered_samples.txt \
--excludeTargets ./extreme_gc_targets.txt --excludeTargets ./low_complexity_targets.txt \
--minTargetSize 10 --maxTargetSize 10000 \
--minMeanTargetRD 10 --maxMeanTargetRD 500 \
--minMeanSampleRD 25 --maxMeanSampleRD 200 \
--maxSdSampleRD 150

xhmm --PCA -r ./DATA.filtered_centered.RD.txt --PCAfiles ./DATA.RD_PCA

xhmm --normalize -r ./DATA.filtered_centered.RD.txt --PCAfiles ./DATA.RD_PCA \
--normalizeOutput ./DATA.PCA_normalized.txt \
--PCnormalizeMethod PVE_mean --PVE_mean_factor 0.7

xhmm --matrix -r ./DATA.PCA_normalized.txt --centerData --centerType sample --zScoreData \
-o ./DATA.PCA_normalized.filtered.sample_zscores.RD.txt \
--outputExcludedTargets ./DATA.PCA_normalized.filtered.sample_zscores.RD.txt.filtered_targets.txt \
--outputExcludedSamples ./DATA.PCA_normalized.filtered.sample_zscores.RD.txt.filtered_samples.txt \
--maxSdTargetRD 30

xhmm --matrix -r ./DATA.RD.txt \
--excludeTargets ./DATA.filtered_centered.RD.txt.filtered_targets.txt \
--excludeTargets ./DATA.PCA_normalized.filtered.sample_zscores.RD.txt.filtered_targets.txt \
--excludeSamples ./DATA.filtered_centered.RD.txt.filtered_samples.txt \
--excludeSamples ./DATA.PCA_normalized.filtered.sample_zscores.RD.txt.filtered_samples.txt \
-o ./DATA.same_filtered.RD.txt

xhmm --discover -p params.txt -r ./DATA.PCA_normalized.filtered.sample_zscores.RD.txt \
    -R ./DATA.same_filtered.RD.txt -c ./DATA.xcnv -a ./DATA.aux_xcnv -s ./DATA

xhmm --genotype -p params.txt -r ./DATA.PCA_normalized.filtered.sample_zscores.RD.txt \
    -R ./DATA.same_filtered.RD.txt -g ./DATA.xcnv -F $ref_fa -v ./DATA.vcf

# junk

Checking if anyone got removed:

In [12]:
%%bash
cd ~/data/multiplex_simplex/xhmm/
ls -ltr DATA.filtered_centered.RD.txt.filtered_samples.txt

-rw-rw---- 1 sudregp sudregp 0 Jan 25 15:37 DATA.filtered_centered.RD.txt.filtered_samples.txt
